In [373]:
import time
import numpy as np
import pandas as pd
import turicreate as tc
import mysql.connector as db
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [356]:
mydb = db.connect(
  host = "localhost",
  user = "root",
  passwd = "",
  database = "ecomm_ml"
)

lineItems = mydb.cursor()
lineItems.execute("SELECT `customer_id`, `product_id`, `quantity` FROM line_items")
lineItems = lineItems.fetchall()

customers = mydb.cursor()
customers.execute("SELECT `id` from `customers`")
customers = customers.fetchall()

In [357]:
lineItems = pd.DataFrame(lineItems,
                         columns=['customer_id', 'product_id', 'quantity'])

In [371]:
customers_list = []
for i in customers:
    customers_list.append(i[0])

In [366]:
# print(lineItems.shape)
# lineItems.head(15)

In [365]:
lineItems = lineItems.groupby(['customer_id', 'product_id'],
                             as_index=False)['quantity'].sum()
lineItems.head(15)

,customer_id,product_id,quantity
0,1,1466095829091,1
1,1,1466095960163,1
2,1,1466096189539,1
3,2,1466095894627,1
4,2,1466095960163,1
5,3,1466095829091,2
6,4,1466095861859,3
7,4,1466095894627,1
8,4,1466095927395,2
9,4,1466096418915,1


In [280]:
# matrix = pd.pivot_table(newItems, values='quantity', index='customer_id', columns='product_id')
# print(matrix.shape)
# matrix.head()

In [278]:
# https://stats.stackexchange.com/questions/70801/how-to-normalize-data-to-0-1-range
# matrix_norm = (matrix-matrix.min())/(matrix.max()-matrix.min())
# print(matrix_norm.shape)
# matrix_norm.head()

In [274]:
# d = matrix_norm.reset_index()
# d.index.names = ['something_new']
# data_norm = pd.melt(d, id_vars=['customer_id'], value_name='something_new').dropna()
# print(data_norm.shape)
# data_norm.head(80)

In [281]:
train, test = train_test_split(lineItems, test_size = .2)
print(train.shape, test.shape)

(11297, 3) (2825, 3)


In [282]:
train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [369]:
# train_data

In [370]:
# test_data

In [351]:
popularity_model = tc.popularity_recommender.create(train_data,
                                                    user_id="customer_id",
                                                    item_id="product_id",
                                                    target="quantity")

Preparing data set.

Data has 11297 observations with 7324 users and 25 items.

Data prepared in: 0.047193s

11297 observations to process; with 25 unique items.

In [375]:
popularity_recommendations = popularity_model.recommend(users=customers_list,
                                                       k=25)

recommendations finished on 1000/9265 queries. users per second: 104537

recommendations finished on 2000/9265 queries. users per second: 178747

recommendations finished on 3000/9265 queries. users per second: 212089

recommendations finished on 4000/9265 queries. users per second: 244828

recommendations finished on 5000/9265 queries. users per second: 269411

recommendations finished on 6000/9265 queries. users per second: 242591

recommendations finished on 7000/9265 queries. users per second: 255596

recommendations finished on 8000/9265 queries. users per second: 272647

recommendations finished on 9000/9265 queries. users per second: 283786

In [376]:
popularity_recommendations.print_rows(50)

+-------------+---------------+--------------------+------+
| customer_id |   product_id  |       score        | rank |
+-------------+---------------+--------------------+------+
|      1      | 1466318520419 | 2.202797202797203  |  1   |
|      1      | 1466096451683 | 1.9411764705882353 |  2   |
|      1      | 1466095894627 | 1.7065637065637065 |  3   |
|      1      | 1466096418915 | 1.7007874015748032 |  4   |
|      1      | 1466095927395 | 1.6109979633401221 |  5   |
|      1      | 1466096189539 | 1.5442715700141443 |  6   |
|      1      | 1466096255075 | 1.411764705882353  |  7   |
|      1      | 1466096353379 | 1.3066666666666666 |  8   |
|      1      | 1466096386147 | 1.3048780487804879 |  9   |
|      1      | 1466096320611 | 1.2489082969432315 |  10  |
|      1      | 1466095829091 | 1.2045769764216365 |  11  |
|      1      | 1466095861859 | 1.1765350877192982 |  12  |
|      1      | 1466096287843 | 1.1466666666666667 |  13  |
|      1      |  15571187078  | 1.108695